经过FFT后有N个元素的序列x经过傅里叶变化生成有N个元素的y
$$ y[k] = \sum_{n=0}^{N-1}{e^{-2\pi j \frac {kn}{N}}}{x[n]}$$
反FFT
$$ x[n] = \frac {1} {N} \sum_{k=0}^{N-1}{e^{2\pi j \frac {kn}{N}}}{y[k]}$$

欧拉公式：
$$e^{\theta j} = cos(\theta) + jsin(\theta)$$

经过FFT后变为复数形式：$$ x + jy$$
$$x + jy == Acos(\theta) + Asin(\theta)j$$
$$Acos(\theta) + Asin(\theta)j == A(cos(\theta) + sin(\theta)j) == Ae^{\theta j}$$
$$A = \sqrt{(x^2 + y^2)}$$
$$\theta == arctan(\frac y x)$$

通常使用中FFT针对的是实数序列进行运算的，这组数的长度N一般都是2的整数次幂，比如：256,512等,后面主要是看这样数据的处理。

<p> 实数序列FFT后有下面规律：
    
- 下标是0 和 N/2虚部是0， 有单独的名字：DC，NC (直流分量，奈奎斯特分量)
- 下标$ 1 => \frac N 2 -1$正的频率部分，升序排列，下标$\frac N 2 + 1 => N - 1$是负频率部分，降序排列（频域是对称的）


In [ ]:
import numpy as np
import scipy

x = np.array([1.0, 2.0, 1.0, -1.0, 1.5, 2.0])

print("np.fft(x):")
print(np.fft.fft(x))
y = scipy.fftpack.rfft(x)
print("scipy.fftpack.rfft(x):")
print(y)
print("np.fft.rfft:")
print(np.fft.rfft(x))
print("scipy.fftpack.ifft:")
yinv = scipy.fftpack.irfft(y)
print(yinv)

print("np.angle:", np.angle(-3.25-0.433012702j, deg=False))
print("np.arctan:", np.arctan(0.433012702/3.25))


print(np.tan(-3.0091380550951676), np.tan(0.1324545984946257))
print("np.angle->[0, pi]:", np.angle(-3.25-0.433012702j, deg=False) + np.pi)


 <p>真正有用的数据就是从$0 => \frac N 2$, 由于0 和 N/2的分量虚部为0，因此只有N个有效值(2 + (N/2 - 1) * 2)
    
* 下标为0的实数表示了时域信号中的直流成分的多少
* 下标为i的复数a+b*j表示时域信号中周期为(N/f * i)的正弦波和余弦波的成分的多少， 其中f为采样率，a表示cos波形的成分，b表示sin波形的成分

In [ ]:
#直流信号：值不随时间变化的信号
import matplotlib.pyplot as plt
x = np.ones(8)
print(x)
x_fft = np.fft.fft(x)
print(x_fft)
plt.plot(x)

In [ ]:
#正弦波信号
#y(t)=A∗sin((2∗π∗f*t)∗t+phase)

frequency = 1
sample_rate = 8

duration = 1 # unit s
amplitude = 10 # 100
phase = 0  # 0 np.pi / 4  -np.pi / 4

num_samples = sample_rate * duration
delta_t = 1.0 / sample_rate

sine_wave = [amplitude * np.sin(2 * np.pi * frequency * (x * delta_t) + phase) for x in range(num_samples)]  #采样

y_fft = np.fft.fft(sine_wave)/len(sine_wave) #/len 每个成分的分量 ？？
print(y_fft) 

import matplotlib.pyplot as plt

mags = np.abs(y_fft)
plt.plot(mags)


In [ ]:
#余弦波信号
#y(t)=A∗cos((2∗π∗f)∗t+phase)

frequency = 1
sample_rate = 8

duration = 1 # unit s
amplitude = 20 # 100
phase = 0

num_samples = sample_rate * duration
delta_t = 1.0 / sample_rate

cos_wave = [amplitude * np.cos(2 * np.pi * frequency * (x * delta_t) + phase) for x in range(num_samples)]  #采样

y_fft = np.fft.fft(cos_wave)/len(cos_wave) #/len 每个成分的分量 ？？
print(y_fft) 

import matplotlib.pyplot as plt

mags = np.abs(y_fft)
plt.plot(mags)

In [ ]:
combine_wave = np.array(sine_wave) + np.array(cos_wave)

y_fft = np.fft.fft(combine_wave)/len(combine_wave) #/len 每个成分的分量 ？？
print(y_fft) 

import matplotlib.pyplot as plt

mags = np.abs(y_fft)
plt.plot(mags)
plt.show()

#注意余弦波的幅度为 对应频率的复数实部的2倍， 正弦波的幅度为 对应频率的复数虚部的-2倍

In [ ]:
plt.plot(sine_wave)
plt.show()
plt.plot(cos_wave)
plt.show()
plt.plot(combine_wave)

<p> <font color=red>注意余弦波的幅度为 对应频率的复数实部的2倍， 正弦波的幅度为 对应频率的复数虚部的-2倍</font>

<p> <font color=red>同频率的正弦波和余弦波通过不同的系数叠加，可以产生同频率的各种相位的余弦波</font>,这样频域的复数理解为：
  
* 复数的模（绝对值）代表了此频率的余弦波的振幅
* 复数的辐角代表了此频率的余弦波的相位

In [ ]:
#多个不同频率余弦波信号合成
#y(t)=A∗cos((2∗π∗f)∗t+phase)

def create_cos_wave(frequency, sample_rate, duration, amplitude, phase):
    num_samples = sample_rate * duration
    delta_t = 1.0 / sample_rate

    cos_wave = [amplitude * np.cos(2 * np.pi * frequency * (x * delta_t) + phase) for x in range(num_samples)]  #采样
    return cos_wave

cos1_wave = create_cos_wave(frequency = 1, sample_rate = 8, duration = 1, amplitude = 10, phase = 0)
cos2_wave = create_cos_wave(frequency = 2, sample_rate = 8, duration = 1, amplitude = 20, phase = 0)
cos3_wave = create_cos_wave(frequency = 3, sample_rate = 8, duration = 1, amplitude = 30, phase = 0)

combined_signal = np.array(cos1_wave) + np.array(cos2_wave) + np.array(cos3_wave)

y_fft = np.fft.fft(combined_signal)/len(combined_signal) #/len 每个成分的分量 ？？
print(y_fft) 

import matplotlib.pyplot as plt

mags = np.abs(y_fft)
plt.plot(mags)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#合成时域波
# 产生size点取样的三角波，其周期为1
def triangle_wave(size):
    x = np.arange(0, 1, 1.0/size)
    y = np.where(x<0.5, x, 0)
    y = np.where(x>=0.5, 1-x, y)
    return x, y

fft_size = 256

# 计算三角波和其FFT
x, y = triangle_wave(fft_size)
plt.plot(y) #使用取样点为x轴
plt.show()
fy = np.fft.fft(y) / fft_size
plt.plot(np.abs(fy)) #使用取样点为x轴
plt.ylim(-0.01, 0.01) #如果不加限制，幅度相差太大，小的值看不到了
#plt.xlim(10, 20)
plt.show()

plt.plot(np.clip(20*np.log10(np.abs(fy[:40])), -120, 120), "o") # 防止0值的log产生过大的值，使用np.clip限制输出
plt.xlabel("frequency")
plt.ylabel("power(dB)")
plt.title("FFT result of triangle wave")
plt.show()

In [ ]:
# 取FFT计算的结果freqs中的前n项进行合成，返回合成结果，计算loops个周期的波形
def fft_combine(freqs, n, loops=1):
    length = len(freqs) * loops
    data = np.zeros(length)
    index = loops  * (2 * np.pi) * np.arange(0, length, 1.0) / length
    for k, p in enumerate(freqs[:n]):
        if k != 0: p *= 2 # 除去直流成分之外，其余的系数都*2
        data += np.real(p) * np.cos(k*index) # 余弦成分的系数为实数部
        data -= np.imag(p) * np.sin(k*index) # 正弦成分的系数为负的虚数部
    return index, data  

plt.figure()
plt.plot(y, label="original triangle", linewidth=2)
for i in [10]: #[0,1,3,5,7,9]
    index, data = fft_combine(fy, i+1, 2)  # 计算两个周期的合成波形
    plt.plot(data, label = "N=%s" % i)
plt.legend()
plt.title("partial Fourier series of triangle wave")
plt.show()

In [ ]:
#对比与三角波，由正余弦信号模拟方形波的收敛速度较慢

def square_wave(size):
    x = np.arange(0, 1, 1.0/size)
    y = np.where(x<0.5, 1.0, 0)
    return x, y

x, y = square_wave(fft_size)
fy = np.fft.fft(y) / fft_size

plt.figure()
plt.plot(y, label="original square", linewidth=2)
for i in [60]: #[0,1,3,5,7,9]
    index, data = fft_combine(fy, i+1, 2)  # 计算两个周期的合成波形
    plt.plot(data, label = "N=%s" % i)
plt.legend()
plt.title("partial Fourier series of square wave")
plt.show()

In [ ]:
plt.plot(np.clip(20*np.log10(np.abs(fy[:40])), -120, 120), "o")

<p> Reference:
  
* https://wizardforcel.gitbooks.io/hyry-studio-scipy/content/19.html
* https://betterexplained.com/articles/an-interactive-guide-to-the-fourier-transform/

<p>小波变换通俗解释 (更好的对非平稳信号进行时频分析)
    
* https://mp.weixin.qq.com/s/fnQGV32QiBv0Zqytkdcxrw

In [ ]:
test = np.arange(9)[1:]
print(test)
np.fft.rfft(test)
print(scipy.fftpack.rfft(test))